<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%98%A4%EB%B2%84%EC%83%98%ED%94%8C%EB%A7%81_%EC%8B%A4%ED%97%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/파이널 프로젝트/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
yogi_df = pd.read_csv(f'{DATA_PATH}yogiyo_reviews_jsi.csv')

yogi_df.shape

(18686, 6)

In [ ]:
# 요기요 데이터

yogi_df.head()

,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,5,5,5,5.0,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,5,5,5,5.0,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,5,5,5.0,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [ ]:
yogi_df['고객리뷰'] = yogi_df.고객리뷰.str.replace('[^a-zA-Z가-힣0-9 .,!?\'\"]', '', regex=True)
yogi_df.고객리뷰

0           사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요
1                        리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!
2                      잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.
3                                      굿 말이 필요 없는 메뉴 였습니다
4                                             맛있어요 양도많구좋음
                               ...                       
18681                                          맛있게 잘 먹었어요
18682           요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다
18683                                배달 빨라요 맛있게 잘 먹었습니당 !
18684                                              잘 먹었어요
18685    식후감 자주 시키는데 항상 요청사항도 잘들어주시고 맛있고 빨라요 앞으로도 자주 시킬게요
Name: 고객리뷰, Length: 18686, dtype: object

In [ ]:
def text_ver_1():
    for i in range(yogi_df['사장댓글'].shape[0]):
        pat = '저희\s+'
        tmp = re.split(pat, yogi_df['사장댓글'][i])
        if len(tmp) == 1:
            print(f'{i}번 "저희" 없어서 패스')
            pass
        else:
            tmp = re.split('의', tmp[1])
            cnt = 0
            for t in tmp:
                if len(t) == 0:
                    pass
                elif t[0] == ' ':
                    print(f'{i}번 수정완료')
                    break
                elif t[0] == '매':
                    print(f'{i}번 "가게명" 없어서 패스')
                    break
                else:
                    t = t + '의'
                    yogi_df['사장댓글'][i] = yogi_df['사장댓글'][i].replace(t, '')
                    cnt += 1
                    if cnt == len(tmp):
                        print(f'{i}번 수정부분 없음')

In [ ]:
def text_ver_2():
    for i in range(yogi_df['사장댓글'].shape[0]):
        pat = '안녕하세요\s+'
        tmp = re.split(pat, yogi_df['사장댓글'][i])
        if len(tmp) == 1:
            print(f'{i}번 "안녕하세요" 없어서 패스')
            pass
        else:
            tmp = re.split('입니다', tmp[1])
            cnt = 0
            for t in tmp:
                if len(t) == 0:
                    pass
                elif t[0] == ' ':
                    print(f'{i}번 수정완료')
                    break
                else:
                    t = t + '입니다'
                    yogi_df['사장댓글'][i] = yogi_df['사장댓글'][i].replace(t, '')
                    cnt += 1
                    if cnt == len(tmp):
                        print(f'{i}번 수정부분 없음')

In [ ]:
def text_ver_3():
    for i in range(yogi_df['사장댓글'].shape[0]):
        pat = '안녕하세요.\s+'
        tmp = re.split(pat, yogi_df['사장댓글'][i])
        if len(tmp) == 1:
            print(f'{i}번 "안녕하세요." 없어서 패스')
            pass
        else:
            tmp = re.split('입니다.', tmp[1])
            cnt = 0
            for t in tmp:
                if len(t) == 0:
                    pass
                elif t[0] == ' ':
                    print(f'{i}번 수정완료')
                    break
                else:
                    t = t + '입니다.'
                    yogi_df['사장댓글'][i] = yogi_df['사장댓글'][i].replace(t, '')
                    cnt += 1
                    if cnt == len(tmp):
                        print(f'{i}번 수정부분 없음')

In [ ]:
def text_remove(text):
    if text == '저희':
        return text_ver_1()
    elif text == '안녕하세요':
        return text_ver_2()
    elif text == '안녕하세요.':
        return text_ver_3()

In [ ]:
def text_remove(text):
    if text == '저희':
        return text_ver_1()
    elif text == '안녕하세요':
        return text_ver_2()
    elif text == '안녕하세요.':
        return text_ver_3()
    else:
        print(f'"{text}"는 실행할 수 없는 단어입니다.')

In [ ]:
check_lst = ['저희', '안녕하세요', '안녕하세요.']

for lst in check_lst:
    text_remove(lst)

0번 "가게명" 없어서 패스
1번 "저희" 없어서 패스
2번 "가게명" 없어서 패스
3번 "가게명" 없어서 패스
4번 수정부분 없음
5번 "가게명" 없어서 패스
6번 "가게명" 없어서 패스
7번 "가게명" 없어서 패스
8번 "가게명" 없어서 패스
9번 "가게명" 없어서 패스
10번 "가게명" 없어서 패스
11번 수정부분 없음
12번 "가게명" 없어서 패스
13번 "가게명" 없어서 패스
14번 "가게명" 없어서 패스
15번 "가게명" 없어서 패스
16번 수정완료
17번 "가게명" 없어서 패스
18번 "가게명" 없어서 패스
19번 "가게명" 없어서 패스
20번 "가게명" 없어서 패스
21번 "저희" 없어서 패스
22번 "저희" 없어서 패스
23번 "저희" 없어서 패스
24번 수정부분 없음
25번 수정부분 없음
26번 "가게명" 없어서 패스
27번 "저희" 없어서 패스
28번 "가게명" 없어서 패스
29번 "가게명" 없어서 패스
30번 "저희" 없어서 패스
31번 "저희" 없어서 패스
32번 수정부분 없음
33번 "가게명" 없어서 패스
34번 "가게명" 없어서 패스
35번 "가게명" 없어서 패스
36번 "가게명" 없어서 패스
37번 "가게명" 없어서 패스
38번 "저희" 없어서 패스
39번 "가게명" 없어서 패스
40번 "저희" 없어서 패스
41번 "가게명" 없어서 패스
42번 "가게명" 없어서 패스
43번 "가게명" 없어서 패스
44번 "가게명" 없어서 패스
45번 수정부분 없음
46번 "가게명" 없어서 패스
47번 "가게명" 없어서 패스
48번 "저희" 없어서 패스
49번 "저희" 없어서 패스
50번 "가게명" 없어서 패스
51번 "가게명" 없어서 패스
52번 "가게명" 없어서 패스
53번 "저희" 없어서 패스
54번 수정부분 없음
55번 "가게명" 없어서 패스
56번 "저희" 없어서 패스
57번 수정부분 없음
58번 "가게명" 없어서 패스
59번 "가게명" 없어서 패스
60번 수정완료
61번 "저희" 없어서 패스
62번 "가게명" 없어서 패스
63번 "가게명"

<ipython-input-6-dafcd7498dad>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yogi_df['사장댓글'][i] = yogi_df['사장댓글'][i].replace(t, '')


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
14004번 "저희" 없어서 패스
14005번 "저희" 없어서 패스
14006번 수정부분 없음
14007번 "저희" 없어서 패스
14008번 "저희" 없어서 패스
14009번 "저희" 없어서 패스
14010번 "저희" 없어서 패스
14011번 "저희" 없어서 패스
14012번 "저희" 없어서 패스
14013번 "저희" 없어서 패스
14014번 수정부분 없음
14015번 "저희" 없어서 패스
14016번 "저희" 없어서 패스
14017번 "저희" 없어서 패스
14018번 "저희" 없어서 패스
14019번 "저희" 없어서 패스
14020번 "저희" 없어서 패스
14021번 "저희" 없어서 패스
14022번 "저희" 없어서 패스
14023번 "저희" 없어서 패스
14024번 "저희" 없어서 패스
14025번 "저희" 없어서 패스
14026번 "저희" 없어서 패스
14027번 "저희" 없어서 패스
14028번 "저희" 없어서 패스
14029번 "저희" 없어서 패스
14030번 "저희" 없어서 패스
14031번 "저희" 없어서 패스
14032번 "저희" 없어서 패스
14033번 "저희" 없어서 패스
14034번 "저희" 없어서 패스
14035번 "저희" 없어서 패스
14036번 "저희" 없어서 패스
14037번 "저희" 없어서 패스
14038번 "저희" 없어서 패스
14039번 "저희" 없어서 패스
14040번 "저희" 없어서 패스
14041번 "저희" 없어서 패스
14042번 "저희" 없어서 패스
14043번 "저희" 없어서 패스
14044번 "저희" 없어서 패스
14045번 "저희" 없어서 패스
14046번 "저희" 없어서 패스
14047번 "저희" 없어서 패스
14048번 "저희" 없어서 패스
14049번 "저희" 없어서 패스
14050번 "저희" 없어서 패스
14051번 "저희" 없어서 패스
14052번 "저희" 없어서 패스
14053번 "저희" 없어서 패스
14054번 "가게명" 없어서 패스
14

<ipython-input-7-44b3ff6218bc>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yogi_df['사장댓글'][i] = yogi_df['사장댓글'][i].replace(t, '')


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
13686번 "안녕하세요." 없어서 패스
13687번 "안녕하세요." 없어서 패스
13688번 "안녕하세요." 없어서 패스
13689번 "안녕하세요." 없어서 패스
13690번 "안녕하세요." 없어서 패스
13691번 "안녕하세요." 없어서 패스
13692번 "안녕하세요." 없어서 패스
13693번 "안녕하세요." 없어서 패스
13694번 "안녕하세요." 없어서 패스
13695번 "안녕하세요." 없어서 패스
13696번 "안녕하세요." 없어서 패스
13697번 "안녕하세요." 없어서 패스
13698번 "안녕하세요." 없어서 패스
13699번 "안녕하세요." 없어서 패스
13700번 "안녕하세요." 없어서 패스
13701번 "안녕하세요." 없어서 패스
13702번 "안녕하세요." 없어서 패스
13703번 "안녕하세요." 없어서 패스
13704번 "안녕하세요." 없어서 패스
13705번 "안녕하세요." 없어서 패스
13706번 "안녕하세요." 없어서 패스
13707번 "안녕하세요." 없어서 패스
13708번 "안녕하세요." 없어서 패스
13709번 "안녕하세요." 없어서 패스
13710번 "안녕하세요." 없어서 패스
13711번 "안녕하세요." 없어서 패스
13712번 "안녕하세요." 없어서 패스
13713번 "안녕하세요." 없어서 패스
13714번 "안녕하세요." 없어서 패스
13715번 "안녕하세요." 없어서 패스
13716번 "안녕하세요." 없어서 패스
13717번 "안녕하세요." 없어서 패스
13718번 "안녕하세요." 없어서 패스
13719번 "안녕하세요." 없어서 패스
13720번 "안녕하세요." 없어서 패스
13721번 "안녕하세요." 없어서 패스
13722번 "안녕하세요." 없어서 패스
13723번 "안녕하세요." 없어서 패스
13724번 "안녕하세요." 없어서 패스
13725번 "안녕하세요." 없어서 패스
13726번 "안녕하세요." 없어서 패스
13727번 "안녕하세요." 없어서 패

In [ ]:
# 정규표현식
yogi_df['사장댓글'] = yogi_df.사장댓글.str.replace('[^a-zA-Z가-힣0-9 .,!?\'\"]', '', regex=True)

# 개인ID -> 고객님 변경
yogi_df['사장댓글'] = yogi_df.사장댓글.str.replace("\w+님", "고객님", regex=True)
yogi_df['replyContent'] = yogi_df.사장댓글.str.replace("\w+ 님", "고객님", regex=True)

yogi_df.사장댓글

0        이렇게 저희 매장 찾아주셔서 감사드립니다!!맛 뿐만 아니라 양도, 서비스도 모두 최...
1        좋은 리뷰 덕분에 기분이 행복해지는 것 고객님께서 맛있다고 말씀 주시니 피로가 싹 ...
2        소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3        수 많은 매장들을 헤치고 와주셔서 감사합니다!!저희 매장의 메뉴를 제대로 즐기신 거...
4        수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 ...
                               ...                        
18681                                감사합니다!!! 다음에도 이용해주세용 
18682                                죄송합니다 !! 다음에도 애용해주세용 
18683                                감사합니다!!! 다음에도 이용해주세용 
18684                                감사합니다!!! 다음에도 이용해주세용 
18685                                감사합니다!!! 다음에도 이용해주세용 
Name: 사장댓글, Length: 18686, dtype: object

In [ ]:
tmp = {
    'content': yogi_df['고객리뷰'],
    'score': yogi_df['별점'],
    'replyContent': yogi_df['사장댓글']
}

df = pd.DataFrame(tmp)
df

,content,score,replyContent
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요,5,"이렇게 저희 매장 찾아주셔서 감사드립니다!!맛 뿐만 아니라 양도, 서비스도 모두 최..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,5,좋은 리뷰 덕분에 기분이 행복해지는 것 고객님께서 맛있다고 말씀 주시니 피로가 싹 ...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,5,수 많은 매장들을 헤치고 와주셔서 감사합니다!!저희 매장의 메뉴를 제대로 즐기신 거...
4,맛있어요 양도많구좋음,5,수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 ...
...,...,...,...
18681,맛있게 잘 먹었어요,5,감사합니다!!! 다음에도 이용해주세용
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,4,죄송합니다 !! 다음에도 애용해주세용
18683,배달 빨라요 맛있게 잘 먹었습니당 !,5,감사합니다!!! 다음에도 이용해주세용
18684,잘 먹었어요,5,감사합니다!!! 다음에도 이용해주세용


In [ ]:
import random
import pickle
import re

wordnet = {}
with open(f"{DATA_PATH}wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)


# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)

	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return

	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-14-0b3a1f8545e8>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [ ]:
df.score.value_counts()

5    17791
4      560
3      179
1       92
2       64
Name: score, dtype: int64

In [ ]:
augmented_replyContent = []
augmented_content = []
augmented_ratings = []
num_aug_1 = 55
num_aug_2 = 80

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    if row["score"] == 1:
        augmented = EDA(row["replyContent"], alpha_sr=0.2, alpha_ri=0.2, alpha_rs=0.2, p_rd=0.1, num_aug=num_aug_1)
        augmented_replyContent.extend(augmented)

        augmented = EDA(row["content"], alpha_sr=0.2, alpha_ri=0.2, alpha_rs=0.2, p_rd=0.1, num_aug=num_aug_1)
        augmented_content.extend(augmented)

        augmented_ratings.extend([row["score"]] * len(augmented))

    elif row["score"] == 2:
        augmented = EDA(row["replyContent"], alpha_sr=0.2, alpha_ri=0.2, alpha_rs=0.2, p_rd=0.1, num_aug=num_aug_2)
        augmented_replyContent.extend(augmented)

        augmented = EDA(row["content"], alpha_sr=0.2, alpha_ri=0.2, alpha_rs=0.2, p_rd=0.1, num_aug=num_aug_2)
        augmented_content.extend(augmented)

        augmented_ratings.extend([row["score"]] * len(augmented))

    # elif row["score"] == 3:
    #     augmented = EDA(row["replyContent"], alpha_sr=0.2, alpha_ri=0.2, alpha_rs=0.2, p_rd=0.1, num_aug=5)
    #     augmented_replyContent.extend(augmented)
    #     augmented_ratings.extend([row["score"]] * len(augmented))

    # elif row["score"] == 4:
    #     augmented = EDA(row["replyContent"], alpha_sr=0.2, alpha_ri=0.2, alpha_rs=0.2, p_rd=0.1, num_aug=5)
    #     augmented_replyContent.extend(augmented)
    #     augmented_ratings.extend([row["score"]] * len(augmented))

    else:
        pass


# sr = 대체, ri 는 삽입, rs는 위치교환, p_rd 는 삭제입니다. num_aug 는 생성할 증강 수입니다

  0%|          | 0/18686 [00:00<?, ?it/s]

In [ ]:
tmp_df = df.copy()
tmp_df.shape

(18686, 3)

In [ ]:
df_augmented = pd.DataFrame({'content': augmented_content, 'replyContent': augmented_replyContent, 'score': augmented_ratings})

tmp_df = pd.concat([tmp_df, df_augmented], ignore_index=True)
tmp_df.shape

(29022, 3)

In [ ]:
tmp_df.score.value_counts()

5    17791
2     5248
1     5244
4      560
3      179
Name: score, dtype: int64

In [ ]:
tmp_df.to_csv(f'{DATA_PATH}yogi_df_over_ver2.csv', index=False)